<a href="https://colab.research.google.com/github/chandra-prakash-94/GenAI/blob/main/video_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Extract required number of still images(frames) from a given video

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import cv2
import os

In [ ]:
# Set number of frames to extract
num_frames = 10

# Open video file
vidcap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/Video_summary/theft_sample2.mp4')


In [ ]:
# Get video properties
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
fps

30

In [ ]:
# Calculate total number of frames present in the input video
frame = 0
while vidcap.isOpened():
    success, image = vidcap.read()
    if success:
      frame += 1
    else:
        break

total_frames = frame
vidcap.release()

In [ ]:
total_frames

338

In [ ]:
#calculate frame interval
frame_interval = int(total_frames/num_frames)

In [ ]:
frame_interval

33

Extract frame for every frame interval

In [ ]:
vidcap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/Video_summary/theft_sample2.mp4')

In [ ]:
# Initialize frame counter
count = 0

# Loop through frames
while vidcap.isOpened():
    success, image = vidcap.read()
    if success:
      if(count%frame_interval==0):
          cv2.imwrite(os.path.join('/content/drive/MyDrive/Colab Notebooks/Video_summary/images_extracted1/', 'frame%d.jpg' %count), image)
          print('Frame {} saved'.format(count))
      count += 1
    else:
        break

vidcap.release()

Frame 0 saved
Frame 33 saved
Frame 66 saved
Frame 99 saved
Frame 132 saved
Frame 165 saved
Frame 198 saved
Frame 231 saved
Frame 264 saved
Frame 297 saved
Frame 330 saved


# 2. List out the file names of extracted images and make a combined file name list

In [ ]:
def list_files_and_make_list(path):
    # List out files from the given path
    files = os.listdir(path)

    # Create a list of file names
    file_names = []
    for file in files:
          file_names.append(file)

    return file_names

# usage:
directory_path = "/content/drive/MyDrive/Colab Notebooks/Video_summary/images_extracted1"
file_names = list_files_and_make_list(directory_path)


In [ ]:
file_names

['frame0.jpg',
 'frame33.jpg',
 'frame66.jpg',
 'frame99.jpg',
 'frame132.jpg',
 'frame165.jpg',
 'frame198.jpg',
 'frame231.jpg',
 'frame264.jpg',
 'frame297.jpg',
 'frame330.jpg']

# 3. Generate caption for each image present in combined list of images using transformers 'image-to-text' model and make a list having captions from each of the images

In [ ]:
from transformers import pipeline

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")


In [ ]:
def generate_captions_for_images(file_names):
    captions = []
    for image in file_names:
        caption = image_to_text("/content/drive/MyDrive/Colab Notebooks/Video_summary/images_extracted1/{}".format(image))
        captions.append(caption[0]['generated_text'])
    return captions


In [ ]:
# usage:
captions = generate_captions_for_images(file_names)
captions

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['a blue and black skateboard sitting on a tile floor ',
 'a blue and black skateboard sitting on a tile floor ',
 'a person standing in a room with a blue and white surfboard ',
 'a person standing in a room with a suitcase ',
 'a man is standing in front of a blue and white kite ',
 'a man is holding a small blue and white kite ',
 'a man is standing in a room with a blue and white t-shirt ',
 'a man is using a pair of scissors to cut a skateboard ',
 'a man holding a cell phone in front of a mirror ',
 'a person standing in a room with a suitcase ',
 'a person is holding a blue and white polka dot polka dot umbrella ']

# 4. Make a combined paragraph from captions generated and summarize the para using transformers 'summarization' model


In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [ ]:
def combine_texts(captions):
    # Join the list of texts into a single paragraph
    combined_paragraph = " ".join(captions)
    return combined_paragraph


In [ ]:
# usage:
combined_paragraph = combine_texts(captions)
combined_paragraph

'a blue and black skateboard sitting on a tile floor  a blue and black skateboard sitting on a tile floor  a person standing in a room with a blue and white surfboard  a person standing in a room with a suitcase  a man is standing in front of a blue and white kite  a man is holding a small blue and white kite  a man is standing in a room with a blue and white t-shirt  a man is using a pair of scissors to cut a skateboard  a man holding a cell phone in front of a mirror  a person standing in a room with a suitcase  a person is holding a blue and white polka dot polka dot umbrella '

In [ ]:
ARTICLE = combined_paragraph
print(summarizer(ARTICLE, max_length=50, min_length=30, do_sample=False))

[{'summary_text': 'A man is using a pair of scissors to cut a skateboard. A person is standing in a room with a suitcase. A man is holding a small blue and white kite.'}]
